In [1]:
#code to load data into pytorch

In [ ]:
from facenet_pytorch import MTCNN
import math
import cv2
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
import torch
import os
import json
import shutil

In [ ]:
#check if GPU is present
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

In [14]:
#load metadata.json file 
#metadata file has videos and corresponding labels "REAL" or "FAKE"
fp=open('metadata.json',)
data=json.load(fp)

In [15]:
#creating data directory 
if not os.path.exists('data'):
    os.makedirs('data')

In [16]:
#creating train_faces directory

if not os.path.exists('data\train'):
    os.makedirs('data\train')

In [17]:
#creating fake and real directories 

if not os.path.exists('data\train\real'):
    os.makedirs('data\train\real')
    
if not os.path.exists('data\train\fake'):
    os.makedirs('data\train\fake')

In [18]:
#creating val_faces directory
if not os.path.exists('data\val'):
    os.makedirs('data\val')

#creating fake and real directories 
if not os.path.exists('data\val\real'):
    os.makedirs('data\val\real')

if not os.path.exists('data\val\fake'):
    os.makedirs('data\val\fake')

In [19]:
#creating validation video file
if not os.path.exists('validation_videos'):
    os.makedirs('validation_videos')

In [ ]:
#training and validation split is 80:20 ratio
# load 20% videos into val folder

import random
x=(os.listdir("train_videos"))

print(len(x))

amt1=0.2*len(x)
amt1=math.ceil(amt1)

In [20]:
i=1
reqd="train_videos"
dest_file=("validation_videos")

while i!=amt1:
    i=i+1
    source=random.choice(os.listdir("train_videos"))
    src=reqd+"\\"+ source
    shutil.copy(src,dest_file)
    os.remove(src)

In [24]:
#loading MTCNN model to detect faces in the frames of the video

mtcnn = MTCNN(margin=20, keep_all=True, post_process=False, device=device)

In [ ]:
#creating training dataset

# split each video into frames and identify faces in each frame
# if the label of the video is "fake" - save the face frame to \data\fake
# if the label of the video is "real" - save the face frame to \data\real

d="train_videos"     # folder where train videos are located



for path in os.listdir(d):
    
    #load video
    v_cap=cv2.VideoCapture("train_videos"+'\\'+path)
    v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    #for every video select every 30th frame
    
    frames = []
    for i in tqdm(range(v_len)):
    
        # Load frame
        success = v_cap.grab()
        if i % 30 == 0:
            success, frame = v_cap.retrieve()
        else:
            continue
        if not success:
            continue
        
        #add to batch
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(Image.fromarray(frame))
        
    file_name = "real" if data[path]['label']=='REAL' else "fake"
    
    
    f="data\train\\" + file_name + "\\"
    frames_paths = [f+ path[0:-4] + str(i) +'.jpg' for i in range(1,len(frames)+1)]
    faces = mtcnn(frames,save_path=frames_paths)



In [ ]:
#creating validation dataset

# split each video into frames and identify faces in each frame
# if the label of the video is "fake" - save the face frame to \data\fake
# if the label of the video is "real" - save the face frame to \data\real

d=r"validation_videos"  



for path in os.listdir(d):
    
    #load video
    v_cap=cv2.VideoCapture("validation_videos"+'\\'+path)
    v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    #for every video, select the 30th frame
    
    frames = []
    for i in tqdm(range(v_len)):
    
        # Load frame
        success = v_cap.grab()
        if i % 30 == 0:
            success, frame = v_cap.retrieve()
        else:
            continue
        if not success:
            continue
        
        #add to batch
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(Image.fromarray(frame))
        
    file_name = "real" if data[path]['label']=='REAL' else "fake"
    
    
    f=r"data\val\\" + file_name + "\\"
    frames_paths = [f+ path[0:-4] + str(i) +'.jpg' for i in range(1,len(frames)+1)]
    faces = mtcnn(frames,save_path=frames_paths)



In [1]:
#create checkpoint file 
f=open("checkpoint.pth","x")

In [2]:
f.close()